<a href="https://colab.research.google.com/github/akshatjain2k/Data-Science-NLP/blob/Amey/MA_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapping the Merger/Acquisition news data

#### https://www.vccircle.com/

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Trail: Scrapping one page

In [ ]:
response = requests.get("https://www.vccircle.com/deal-type/m-a/all/1")
soup = BeautifulSoup(response.text)
news = soup.find_all("div",class_='newsCard_content__Glu72')

for new in news:
  try:
    title = new.find("h4", class_ = "listingPage_list-heading__5XeLA")
    link = title.find('a')['href']
    date = new.find('li').text
    print(title.text)
  except:
    break


Lifelong Group-led consortium snaps up GoMechanic
Kae Capital-backed LoanTap snaps up Unofin to boost healthcare lending play
Jindal Stainless buys $157-mn stake in Indonesian firm to secure nickel supply
Bessemer-backed Medi Assist buys Raksha Health; Sportzcraazy snaps up Kabaddi Adda
First Citizens Bank agrees to buy Silicon Valley Bank
VC-backed Redcliffe buys diagnostic lab chain in first acquisition
Why dealmakers expect M&A activity to pick up despite facing 'brick wall'
Unacademy eyes stake in Byju's-owned Aakash
PhysicsWallah strikes maiden overseas bet with UAE firm Knowledge Planet
Times Group's OTT platform MX Player may go for a song


[]

## Scrapping the link page for news body

In [ ]:
response = requests.get("https://www.vccircle.com/lifelonggroup-led-consortium-snaps-up-gomechanic")
soup2 = BeautifulSoup(response.text)
body = soup2.find_all("p", class_ = "")
para =[]
for i in body:
  para.append(i.text)
s = ''.join(para)

s

"A consortium led by auto component maker Lifelong Group on Wednesday said it has bought beleaguered car workshop and auto spare parts platform GoMechanic, nearly two months after the latter’s co-founders admitted to falsifying figures to investors.The financial details of the deal were not disclosed.Lifelong Group will become the majority shareholder under Gurugram-based car care startup Servizzy, marking a new phase for the GoMechanic business, it said in a statement.The acquisition of GoMechanic business aligns with the group's strategic vision of synergizing its proven expertise in the automotive industry, said Lifelong, which caters to major players such as Hero Moto Corp, General Motors, Arvin Meritor, and Stanley Black & Decker. \xa0The sale was initiated by the board and shareholders of GoMechanic, with support from venture debt investor Stride Ventures, in response to recent financial difficulties at the Gurgaon-based company. The Servizzy consortium, led by the Lifelong Group

## Loop for scrapping 100 pages

In [ ]:
def scrapper():
    date_ = []
    title_ = []
    link_ = []
    body_news_ = []

    for i in range(1,100):
            try:
                response = requests.get(f"https://www.vccircle.com/deal-type/m-a/all/{i}")
            except:
                print("link not working")
            soup = BeautifulSoup(response.text)
            news = soup.find_all("div", class_ = "newsCard_content__Glu72")
            for new in news:
                title = new.find("h4", class_ = "listingPage_list-heading__5XeLA")
                try:
                  link = title.find('a')['href']
                except:
                  pass
                date = new.find("li")
                
                
                ### for title 
                try:             
                    title_text = title.text
                    title_.append(title_text)
                except:
                    title_.append("None")
                    
                    
                    
                ### for links    
                try:             
                    
                    l = f"https://www.vccircle.com/{link}"
                    link_.append(l)
                    #print(l)
                    
                    
                except:
                    link_.append("None")
                    
                    
                ### for date
                try:             
                    
                    d = date.text
                    
                    date_.append(d)
                    #print(d)
                    
                except:
                    date_.append("None")
                    
                    
                ### code for the next page deatiled news
                try:    
                    resp = requests.get(l)
                    n_url = resp.text
                except:
                    print("Link not working for next page")
                    
                soup2 = BeautifulSoup(n_url)
                body = soup2.find_all("p", class_ = "")

                
                ### for body of the news in next page
                try:
                    para =[]
                    for i in body:
                      para.append(i.text)
                    body = ''.join(para)
                    
                    body_news_.append(body)
                    #print(body.text)
                except:
                    body_news_.append("None")
    return date_, title_, link_, body_news_

In [ ]:
date_, title_, link_, body_news_ = scrapper()

In [ ]:
print(len(date_))
print(len(title_))
print(len(link_))
print(len(body_news_))

16
16
16
16


In [ ]:
# Converting into DataFrame
dict_ = {"Date_Time":date_, "Title":title_, "Full_News":body_news_, "Links": link_, "Category": "Merger/Acquisition"}
df = pd.DataFrame(dict_)

In [ ]:
df.head()

,Date_Time,Title,Full_News,Links,Category
0,29 March,Lifelong Group-led consortium snaps up GoMechanic,A consortium led by auto component maker Lifel...,https://www.vccircle.com//lifelonggroup-led-co...,Merger/Acquisition
1,29 March,Kae Capital-backed LoanTap snaps up Unofin to ...,Pune-based digital lending platform LoanTap Fi...,https://www.vccircle.com//kaecapital-backed-lo...,Merger/Acquisition
2,28 March,Jindal Stainless buys $157-mn stake in Indones...,India's Jindal Stainless Ltd on Tuesday said i...,https://www.vccircle.com//jindalstainless-buys...,Merger/Acquisition
3,27 March,Bessemer-backed Medi Assist buys Raksha Health...,Health insurance third-party administrator Med...,https://www.vccircle.com//bessemerbacked-medi-...,Merger/Acquisition
4,27 March,First Citizens Bank agrees to buy Silicon Vall...,First Citizens BancShares Inc said on Monday i...,https://www.vccircle.com//firstcitizens-bank-a...,Merger/Acquisition


## Dropping the none values

In [ ]:
# replace none in df with nan
df.replace("None", float("NaN"), inplace = True)

In [ ]:
df.isna().sum()

Date_Time    6
Title        6
Full_News    0
Links        0
Category     0
dtype: int64

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.isna().sum()

Date_Time    0
Title        0
Full_News    0
Links        0
Category     0
dtype: int64

In [ ]:
# save dataframe
df.to_csv('Stocks_news.csv', index=False)

In [ ]:
df.head()

,Date_Time,Title,Full_News,Links,Category
0,29 March,Lifelong Group-led consortium snaps up GoMechanic,A consortium led by auto component maker Lifel...,https://www.vccircle.com//lifelonggroup-led-co...,Merger/Acquisition
1,29 March,Kae Capital-backed LoanTap snaps up Unofin to ...,Pune-based digital lending platform LoanTap Fi...,https://www.vccircle.com//kaecapital-backed-lo...,Merger/Acquisition
2,28 March,Jindal Stainless buys $157-mn stake in Indones...,India's Jindal Stainless Ltd on Tuesday said i...,https://www.vccircle.com//jindalstainless-buys...,Merger/Acquisition
3,27 March,Bessemer-backed Medi Assist buys Raksha Health...,Health insurance third-party administrator Med...,https://www.vccircle.com//bessemerbacked-medi-...,Merger/Acquisition
4,27 March,First Citizens Bank agrees to buy Silicon Vall...,First Citizens BancShares Inc said on Monday i...,https://www.vccircle.com//firstcitizens-bank-a...,Merger/Acquisition


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date_Time  10 non-null     object
 1   Title      10 non-null     object
 2   Full_News  10 non-null     object
 3   Links      10 non-null     object
 4   Category   10 non-null     object
dtypes: object(5)
memory usage: 480.0+ bytes
